# Netflix & Top Grossing Films ETL

In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
!pip install psycopg2

## Extract

In [2]:
csv_file = "netflix_titles.csv"
netflix_df = pd.read_csv(csv_file)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
csv_file_1 = "imdb_top_1000.csv"
imdb_df = pd.read_csv(csv_file_1)
imdb_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


## Transform

In [4]:
# Clean netflix_df
netflix_df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [5]:
# We want to drop show_id, cast, country, date_added
del netflix_df['show_id']
del netflix_df['cast']
del netflix_df['country']
del netflix_df['date_added']
del netflix_df['rating']

In [6]:
# Rename columns 
netflix_df = netflix_df.rename(columns={"type":"genre"})
netflix_df = netflix_df.rename(columns={"duration":"runtime"})
netflix_df = netflix_df.rename(columns={"description":"overview"})

In [7]:
# Clean imdb_df
imdb_df.columns

Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

In [8]:
# We want to drop Poster_link, Star1, Star2, Star3, Star4
del imdb_df['Poster_Link']
del imdb_df['Star1']
del imdb_df['Star2']
del imdb_df['Star3']
del imdb_df['Star4']
del imdb_df['Certificate']

In [9]:
# Rename columns 
imdb_df = imdb_df.rename(columns={"Series_Title":"title"})
imdb_df = imdb_df.rename(columns={"Released_Year":"release_year"})
imdb_df = imdb_df.rename(columns={"Runtime":"runtime"})
imdb_df = imdb_df.rename(columns={"Genre":"genre"})
imdb_df = imdb_df.rename(columns={"Overview":"overview"})
imdb_df = imdb_df.rename(columns={"Director":"director"})

In [10]:
# Merge dfs
netflix_imdb1 = pd.concat([netflix_df, imdb_df])
netflix_imdb1

,genre,title,director,release_year,runtime,listed_in,overview,IMDB_Rating,Meta_score,No_of_Votes,Gross
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,2020,90 min,Documentaries,"As her father nears the end of his life, filmm...",NaN,NaN,NaN,NaN
1,TV Show,Blood & Water,NaN,2021,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,NaN,NaN,NaN
2,TV Show,Ganglands,Julien Leclercq,2021,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,NaN,NaN,NaN
3,TV Show,Jailbirds New Orleans,NaN,2021,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN,NaN,NaN,NaN
4,TV Show,Kota Factory,NaN,2021,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,"Comedy, Drama, Romance",Breakfast at Tiffany's,Blake Edwards,1961,115 min,NaN,A young New York socialite becomes interested ...,7.6,76.0,166544.0,NaN
996,"Drama, Western",Giant,George Stevens,1956,201 min,NaN,Sprawling epic covering the life of a Texas ca...,7.6,84.0,34075.0,NaN
997,"Drama, Romance, War",From Here to Eternity,Fred Zinnemann,1953,118 min,NaN,"In Hawaii in 1941, a private is cruelly punish...",7.6,85.0,43374.0,"30,500,000"
998,"Drama, War",Lifeboat,Alfred Hitchcock,1944,97 min,NaN,Several survivors of a torpedoed merchant ship...,7.6,78.0,26471.0,NaN


In [11]:
netflix_imdb1.loc[netflix_imdb1['release_year'] == 'PG']

,genre,title,director,release_year,runtime,listed_in,overview,IMDB_Rating,Meta_score,No_of_Votes,Gross
966,"Adventure, Drama, History",Apollo 13,Ron Howard,PG,140 min,NaN,NASA must devise a strategy to return Apollo 1...,7.6,77.0,269197.0,"173,837,933"


In [12]:
netflix_imdb1.loc[966, 'release_year'] = 0

In [13]:
netflix_imdb1

,genre,title,director,release_year,runtime,listed_in,overview,IMDB_Rating,Meta_score,No_of_Votes,Gross
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,2020,90 min,Documentaries,"As her father nears the end of his life, filmm...",NaN,NaN,NaN,NaN
1,TV Show,Blood & Water,NaN,2021,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,NaN,NaN,NaN
2,TV Show,Ganglands,Julien Leclercq,2021,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,NaN,NaN,NaN
3,TV Show,Jailbirds New Orleans,NaN,2021,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN,NaN,NaN,NaN
4,TV Show,Kota Factory,NaN,2021,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,"Comedy, Drama, Romance",Breakfast at Tiffany's,Blake Edwards,1961,115 min,NaN,A young New York socialite becomes interested ...,7.6,76.0,166544.0,NaN
996,"Drama, Western",Giant,George Stevens,1956,201 min,NaN,Sprawling epic covering the life of a Texas ca...,7.6,84.0,34075.0,NaN
997,"Drama, Romance, War",From Here to Eternity,Fred Zinnemann,1953,118 min,NaN,"In Hawaii in 1941, a private is cruelly punish...",7.6,85.0,43374.0,"30,500,000"
998,"Drama, War",Lifeboat,Alfred Hitchcock,1944,97 min,NaN,Several survivors of a torpedoed merchant ship...,7.6,78.0,26471.0,NaN


In [14]:
netflix_imdb1.loc[netflix_imdb1['title'] == 'Apollo 13']

,genre,title,director,release_year,runtime,listed_in,overview,IMDB_Rating,Meta_score,No_of_Votes,Gross
966,"Adventure, Drama, History",Apollo 13,Ron Howard,0,140 min,NaN,NASA must devise a strategy to return Apollo 1...,7.6,77.0,269197.0,"173,837,933"


In [15]:
# Change null values
value = "not currently in IMDB top 1000"

netflix_imdb1['IMDB_Rating'] = netflix_imdb1['IMDB_Rating'].replace(np.nan, value)
netflix_imdb1['Meta_score'] = netflix_imdb1['Meta_score'].replace(np.nan, value)
netflix_imdb1['No_of_Votes'] = netflix_imdb1['No_of_Votes'].replace(np.nan, value)
netflix_imdb1['Gross'] = netflix_imdb1['Gross'].replace(np.nan, value)

netflix_imdb1.head()

,genre,title,director,release_year,runtime,listed_in,overview,IMDB_Rating,Meta_score,No_of_Votes,Gross
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,2020,90 min,Documentaries,"As her father nears the end of his life, filmm...",not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
1,TV Show,Blood & Water,NaN,2021,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
2,TV Show,Ganglands,Julien Leclercq,2021,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
3,TV Show,Jailbirds New Orleans,NaN,2021,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
4,TV Show,Kota Factory,NaN,2021,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000


In [16]:
from settings import key

In [17]:
# Load
protocol = "postgresql"
username = "postgres"
host = 'localhost'
port = 5432
database_name = "Netflixmovies_db"

In [18]:
rds_connection_string = f'{protocol}://{username}:{key}@{host}:{port}/{database_name}'


In [19]:
engine = create_engine(rds_connection_string)


In [20]:
engine.table_names()


C:\Users\chisi\AppData\Local\Temp/ipykernel_26692/509743474.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['movies_db']

##  Load DataFrames into database

In [21]:
 netflix_imdb1.to_sql(name='movies_db', con=engine, if_exists='append', index=False)

## Confirming data

In [22]:
pd.read_sql_query('select * from movies_db', con=engine).head()

,genre,title,director,release_year,runtime,listed_in,overview,IMDB_Rating,Meta_score,No_of_Votes,Gross
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,2020,90 min,Documentaries,"As her father nears the end of his life, filmm...",not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
1,TV Show,Blood & Water,None,2021,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
2,TV Show,Ganglands,Julien Leclercq,2021,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
3,TV Show,Jailbirds New Orleans,None,2021,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
4,TV Show,Kota Factory,None,2021,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000,not currently in IMDB top 1000
